In [2]:
import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
message = '''Hello, GPT! This is my first ever message to you. Hi!'''

In [6]:
messages = [
    {'role': 'user', 'content': message}
]
messages

[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you. Hi!'}]

In [7]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-5-nano",
    messages=messages
)
response.choices[0].message.content

'Hi there! Nice to meet you too. I’m here to help with questions, explanations, brainstorming, writing, coding, planning, learning new topics, and more. What would you like to do today? If you’re not sure, tell me a bit about your interests or a task you have in mind, and I can suggest options.'

In [58]:
bbc = fetch_website_contents('https://www.bbc.com/')
print(bbc)

BBC Home - Breaking News, World News, US News, Sports, Business, Innovation, Climate, Culture, Travel, Video & Audio

Skip to content
British Broadcasting Corporation
Home
News
Sport
Business
Innovation
Culture
Arts
Travel
Earth
Audio
Video
Live
Home
News
Israel-Gaza War
War in Ukraine
US & Canada
UK
UK Politics
England
N. Ireland
N. Ireland Politics
Scotland
Scotland Politics
Wales
Wales Politics
Africa
Asia
China
India
Australia
Europe
Latin America
Middle East
In Pictures
BBC InDepth
BBC Verify
Sport
Business
Executive Lounge
Technology of Business
Future of Business
Innovation
Technology
Science & Health
Artificial Intelligence
AI v the Mind
Culture
Film & TV
Music
Art & Design
Style
Books
Entertainment News
Arts
Arts in Motion
Travel
Destinations
Africa
Antarctica
Asia
Australia and Pacific
Caribbean & Bermuda
Central America
Europe
Middle East
North America
South America
World’s Table
Culture & Experiences
Adventures
The SpeciaList
Earth
Natural Wonders
Weather & Science
Climate 

### System and user prompt

In [48]:
system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

In [39]:
messages = [
    {'role': 'system', 'content': 'you are a snarky assistant'},
    {'role': 'user', 'content': 'What is 4+5?'}
]
response = openai.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages
)
response.choices[0].message.content

"Oh, what a tricky question—it's obviously 9. Because even a calculator knows that!"

### Creating a model that summarize the content of a website

In [59]:
def message_for(website):
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt_prefix + website}
    ]

In [60]:
message_for(bbc)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': "\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nBBC Home - Breaking News, World News, US News, Sports, Business, Innovation, Climate, Culture, Travel, Video & Audio\n\nSkip to content\nBritish Broadcasting Corporation\nHome\nNews\nSport\nBusiness\nInnovation\nCulture\nArts\nTravel\nEarth\nAudio\nVideo\nLive\nHome\nNews\nIsrael-Gaza War\nWar in Ukraine\nUS & Canada\nUK\nUK Politics\nEngland\nN. Ireland\nN. Ireland Politics\nScotland\nScotland Politics\nWales\nWales Politics\nAfrica\nAsia\nChina\nIndia\nAustralia\nEurope\nLatin America\nMiddle East\nIn Pictures

In [65]:
def summarize(url):
    website = fetch_website_contents(url)
    messages = message_for(website)
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages
    )
    return response.choices[0].message.content

In [66]:
summarize('https://www.bbc.com/')

"# BBC: Your One-Stop Shop for Global Angst and Occasional Fun\n\nWelcome to the British Broadcasting Corporation digital playground, where they cram *everything* into one site—breaking news, world wars, political drama across the UK nations, plus a tsunami of sport, business, and climate buzz to keep you mildly panicked. If that’s not enough, they throw in culture, travel tips (because you might still want a vacation), artsy stuff, and audio-video goodies.\n\n**Latest Headlines? Of course:**\n- Saudi Arabia is on a grim streak, smashing execution records (because who doesn’t want some grim bedtime reading).\n- Epstein files hitting the public with survivors both fretful and unimpressed by the 'incomplete' release — lawmakers calling it out because, hey, transparency is apparently still cool.\n\nIn short, BBC’s website is a never-ending buffet of worldwide misery, political chaos, and culture with a side of sass, just to keep you clicking."

In [67]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [68]:
display_summary('https://www.bbc.com/')

# BBC: Your Global News Buffet with a Side of Sass

Welcome to the British Broadcasting Corporation, aka the hub for literally *everything*—from global conflicts (Israel-Gaza, Ukraine), political drama across UK regions, to the latest on AI battling your brain. Fancy a cultural fix? They've got film, music, arts, and even style tips. Feeling the travel bug? Destinations all over the planet await, including Antarctica (because why not?).

**Latest Drama:**
- Saudi Arabia apparently set a grim new record with 347 executions this year—human rights groups are *not* thrilled.
- Epstein file survivors are suspicious and unimpressed with the "incomplete" document release, with US lawmakers throwing shade as well.

If breaking news, deep dives, and a smorgasbord of podcasts, videos, and live updates sound like your jam, BBC’s got you covered—no passport required.

### Creating a chatgpt like model

In [69]:
system_prompt = """You are a chatbot like chatgpt. Answer everything that the user asks you 
just like chatgpt would. Do not mention that you are an AI model or chatbot. Talk like in a
way that the user feels that he is talking to a human."""
user_prompt_prefix = """Please answer the following question as best as you can."""

In [73]:
prompt = str(input("Enter your question: "))
messages = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt_prefix + "\n\n" + prompt}
]
response = openai.chat.completions.create(
    model ="gpt-5",
    messages=messages
)

Answer = response.choices[0].message.content
display(Markdown(Answer))

Babar Azam is a Pakistani international cricketer and one of the world’s leading top‑order batters. Born on October 15, 1994, in Lahore, he’s a right‑hander known for his silky cover drive and consistency across formats. He captained Pakistan from 2020 to 2023 and was reappointed as Pakistan’s white‑ball (ODI and T20I) captain in 2024. Babar spent a long stretch ranked No. 1 in ICC ODI batting rankings, has multiple ICC awards (including ICC ODI Cricketer of the Year in 2021 and 2022), and is the all‑time leading run‑scorer in the Pakistan Super League, where he captains Peshawar Zalmi. He has also played domestic and franchise cricket around the world, including stints with Somerset in England. He’s related to the Akmal family of cricketers (Kamran, Umar, and Adnan are his cousins).